## 编写一个程序, get_response(saying, response_rules)输入是一个字符串 + 我们定义的 rules，例如上边我们所写的 pattern， 输出是一个回答。

In [1]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [2]:
def pat_match(pattern, saying):
    if not pattern or not saying: return []
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])

In [3]:
def pat_to_dict(patterns):
    return {k: v for k, v in patterns}

In [4]:
def subsitite(rule, parsed_rules):
    if not rule: return []
    
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

In [5]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

In [6]:
def get_response(saying, rules=defined_patterns):
    """" please implement the code, to get the response as followings:
    
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """
    for key in rules.keys():
        got_patterns = pat_match(key.split(), saying.split())
        if got_patterns!=[]: break
    
    return ' '.join(subsitite(rules[key][0].split(), pat_to_dict(got_patterns)))

In [7]:
get_response("My mother told me something")

'Talk about more about your mother'

## 改写以上程序，将程序变成能够支持中文输入的模式。 提示: 你可以需用用到 jieba 分词

## 多设计一些模式，让这个程序变得更好玩，多和大家交流，看看大家有什么好玩的模式

In [95]:
import jieba
import random

In [96]:
def is_chinese(char):
    if u'\u4e00' <= char <= u'\u9fff':
            return True
    else:
        return False
    
def contain_chinese(check_str):
    for ch in check_str:
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
    return False

In [97]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() or is_chinese(a) for a in pattern[2:])

In [162]:
fail = [True, None]

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        if match==None: return fail
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

In [166]:
pat_match_with_seg("?*x hello ?*yield".split(),"AI, hello !".split())

[('?x', ['AI,']), ('?yield', ['!'])]

In [164]:
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
        if rest[0] == token and (is_match(rest[1:], saying[(i + 1):]) or is_pattern_segment(rest[1])):
            return (seg_pat, saying[:i]), i
    
    return None,0

def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not saying:
        return False
    if not all(a.isalpha() or is_chinese(a) for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

In [100]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [101]:
def subsitite(rule, parsed_rules):
    if not rule: return []
    
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

In [102]:
def split_chinese(ch_str):
    segs=jieba.cut(ch_str)
    return list(segs)
    
def split_str(words):
    split_word=[]
    if not words: return []
    
    if not contain_chinese(words):
        return words.split()
    
    ch=''
    en=''
    for i,char in enumerate(words):
        if is_chinese(char) and en!='':
            return en.split()+split_str(words[i:])
        if not is_chinese(char) and ch!='':
            return split_chinese(ch)+split_str(words[i:])
        if is_chinese(char):
            ch+=char
            continue
        if not is_chinese(char):
            en+=char
            continue
    
    if ch!='':
        return split_chinese(ch)

In [111]:
split_str('你总是这样')

['你', '总是', '这样']

In [118]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}

In [161]:
def get_response(saying, rules):
    split_saying=split_str(saying)
    
    match=0
    for key in rules.keys():
        if key=='?*x': continue
        
        split_key=split_str(key) 
        got_patterns = pat_match_with_seg(split_key, split_saying)
        if got_patterns[-1]!=None: 
            match=1
            break
    
    if match==0: 
        key='?*x'
        split_key=split_str(key)        
        got_patterns = pat_match_with_seg(split_key, split_saying)
        
    returnwords=split_str(random.choice(rules[key]))
    
    if contain_chinese(saying):
        return ''.join(subsitite(returnwords, pat_to_dict(got_patterns)))
    
    return ' '.join(subsitite(returnwords, pat_to_dict(got_patterns)))

In [129]:
get_response("你总是这样",rule_responses)

'你具体是说哪一次？'

In [153]:
get_response("我喜欢你",rule_responses)

'喜欢你的哪里？'

In [155]:
get_response("我讨厌苹果",rule_responses)

'你不想要苹果吗？'

In [133]:
get_response("bot你是人吗",rule_responses)

'你要是喜欢，我就会是人'

In [160]:
get_response("AI, I feel not so good",rule_responses)

'What other feelings do you have?'

## 问题4
### 这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？
优点：运行速度较快，response较合理
缺点：response较单一，需人为设计答案
改进：使用日常对话数据训练模型，使其能够学习出rules
### 什么是数据驱动？数据驱动在这个程序里如何体现？
数据驱动是程序能通过输入来获得必要的信息，对于不同的输入可得到目标的输出。在这个程序中输入不同的rules均可获得合理的response
### 数据驱动与 AI 的关系是什么？
数据驱动是AI的一个特征